In [ ]:
! pip install numpy pandas seaborn scikit-learn joblib neattext torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import neattext as nt
import neattext.functions as nfx
import joblib
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from transformers import pipeline

# Load Pre-Trained BERT Model (Offline)
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-go-emotion", device=-1, return_all_scores=True)

data = {
    "text": [
        # Happy
        "I am so happy today!", "What a wonderful day!", "I love my life!",
        "I feel blessed!", "I’m in the best mood ever!",
        # Sad
        "I feel miserable.", "Life is so tough.", "I am extremely sad today.",
        "This is heartbreaking.", "I feel like crying.",
        # Angry
        "This makes me furious!", "I hate when this happens!", "I am so mad right now!",
        "This is extremely frustrating!", "I feel enraged.",
        # Fear
        "I am scared of the dark.", "This is terrifying!", "I feel so anxious right now.",
        "This makes me nervous.", "I feel uneasy about this.",
        # Surprise
        "That was an amazing surprise!", "Wow! I didn’t expect that!",
        "I can’t believe this happened!", "That was shocking!", "This blew my mind!",
        # Disgust
        "I feel disgusted by this behavior.", "This is revolting!",
        "This makes me sick!", "This is gross!", "I can’t stand this!",
        # Excited
        "I can't wait for the trip!", "I feel so excited!", "This is thrilling!",
        "I am looking forward to this!", "What an amazing experience!",
        # Neutral
        "I feel neutral about it.", "This is okay.", "I don't really mind this.",
        "I have no strong opinion.", "It’s just another day."
    ],
    "emotion": [
        "happy", "happy", "happy", "happy", "happy",
        "sad", "sad", "sad", "sad", "sad",
        "angry", "angry", "angry", "angry", "angry",
        "fear", "fear", "fear", "fear", "fear",
        "surprise", "surprise", "surprise", "surprise", "surprise",
        "disgust", "disgust", "disgust", "disgust", "disgust",
        "excited", "excited", "excited", "excited", "excited",
        "neutral", "neutral", "neutral", "neutral", "neutral"
    ]
}


df = pd.DataFrame(data)

# Preprocess Text
df["clean_text"] = df["text"].apply(nfx.remove_stopwords).apply(nfx.remove_special_characters)

# Feature Extraction (TF-IDF)
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=3000, stop_words="english")
X = vectorizer.fit_transform(df["clean_text"])
y = df["emotion"]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train Naïve Bayes Model
nb_model = MultinomialNB(alpha=0.5)
nb_model.fit(X_train, y_train)

# Predict on Test Set
y_pred = nb_model.predict(X_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
#print(f"Naïve Bayes Model Accuracy: {accuracy:.2f}")

# Save Model & Vectorizer
joblib.dump(nb_model, "emotion_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

# Function to Predict Emotion
def predict_emotion(text, method="naive_bayes"):
    text = nfx.remove_stopwords(nfx.remove_special_characters(text))

    if method == "naive_bayes":
        model = joblib.load("emotion_model.pkl")
        vectorizer = joblib.load("vectorizer.pkl")
        X_input = vectorizer.transform([text])
        return model.predict(X_input)[0]

    elif method == "bert":
      prediction = emotion_classifier(text, return_all_scores=True)
      scores = prediction[0]  #scores for all emotion
      top_emotion = max(scores, key=lambda x: x['score'])  # the highest confidence emotion
      return top_emotion['label']


if __name__ == "__main__":
    while True:
        user_text = input("\nEnter a sentence (or type 'exit' to quit): ")
        if user_text.lower() == "exit":
            break

        print(f"Naïve Bayes Prediction: {predict_emotion(user_text, method='naive_bayes')}")
        print(f"BERT Prediction: {predict_emotion(user_text, method='bert')}")
        df["emotion"].value_counts().plot(kind="bar")


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



Enter a sentence (or type 'exit' to quit): i am thrilled to learn today


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Naïve Bayes Prediction: happy
BERT Prediction: neutral

Enter a sentence (or type 'exit' to quit): i am sad that i wont get free time


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


...

Naïve Bayes Prediction: neutral
BERT Prediction: neutral

Enter a sentence (or type 'exit' to quit): i am excited for the trip


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Naïve Bayes Prediction: excited
BERT Prediction: excitement

Enter a sentence (or type 'exit' to quit): i feel enraged at the actions of those without empathy


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Naïve Bayes Prediction: angry
BERT Prediction: neutral
